In [ ]:
!apt-get -q install poppler-utils
!pip install -q pdf2image langchain langchain-core langchain-community langchain-google-genai

In [ ]:
import os
import io
import base64
from PIL import Image
from pdf2image import convert_from_bytes
from google.colab import files

import langchain_google_genai as genai
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Replace "YOUR_API_KEY" with your actual Gemini API key
GOOGLE_API_KEY = ""
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print("API key configuration complete!")

In [ ]:
model = genai.ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.8,
    convert_system_message_to_human=True,
    max_output_tokens=8192
)

parser = StrOutputParser()

In [53]:
def convert_pdf_to_base64_image(pdf_file):
    images = convert_from_bytes(pdf_file.read())
    first_page = images[0]
    img_byte_arr = io.BytesIO()
    first_page.save(img_byte_arr, format='JPEG')
    img_data = base64.b64encode(img_byte_arr.getvalue()).decode()
    return f"data:image/jpeg;base64,{img_data}"

In [ ]:
HR_REVIEW_PROMPT = """
You are a senior HR specialist. Carefully analyze the resume image and compare it with the provided job description.

Respond with:
1. Whether the candidate fits the job.
2. Strengths and weaknesses in the resume.
3. Suggestions to improve the resume.

Resume Image (Base64): {resume_image}
Job Description: {job_description}
"""
MATCH_PROMPT = """
You are an ATS (Applicant Tracking System) resume scanner. Compare the given resume and job description.

 Output Format (Important):
Match Percentage: <percentage>
Missing Keywords: <comma-separated keywords>
Final Evaluation: <your reasoning>

Use only the format above. Do not add extra explanation or heading.

Resume Image (Base64): {resume_image}
Job Description: {job_description}
"""


In [55]:
def evaluate_resume(pdf_file, job_description, mode="HR Review"):
    resume_image = convert_pdf_to_base64_image(pdf_file)
    prompt_template = ChatPromptTemplate.from_template(
        HR_REVIEW_PROMPT if mode == "HR Review" else MATCH_PROMPT
    )
    chain = prompt_template | model | parser
    result = chain.invoke({
        "resume_image": resume_image,
        "job_description": job_description
    })
    return result

In [57]:
uploaded = files.upload()  # Upload resume PDF manually
pdf_filename = list(uploaded.keys())[0]
print("Please enter the job description for evaluation:")
job_description = input("Job Description: ")
with open(pdf_filename, "rb") as f:
    result = evaluate_resume(f, job_description, mode="Match Percentage")
    print("\n📋 Evaluation Result:\n")
    print(result)


Saving image.pdf to image (2).pdf
Please enter the job description for evaluation:
Job Description: machine learning


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



📋 Evaluation Result:

Match Percentage: 3%
Missing Keywords: machine, learning
Final Evaluation: The resume is a low quality image that contains no relevant information.


In [52]:
pdf_filenames = []
results = []

num_resumes = 3

print(f"\n📤 Upload {num_resumes} resumes (one by one)...")
for i in range(1, num_resumes + 1):
    print(f"\n Upload resume {i}:")
    uploaded = files.upload()
    pdf_filename = list(uploaded.keys())[0]
    pdf_filenames.append(pdf_filename)
print("\n📝 Enter the job description once:")
job_description = input("Job Description: ")
for pdf_filename in pdf_filenames:
    print(f"\nEvaluating: {pdf_filename}")
    with open(pdf_filename, "rb") as f:
        try:
            raw_result = evaluate_resume(f, job_description, mode="Match Percentage")
        except Exception as e:
            print(f" Error while processing {pdf_filename}: {e}")
            continue
    match, keywords, summary = "", "", ""
    for line in raw_result.splitlines():
        if line.startswith("Match Percentage:"):
            match = line.replace("Match Percentage:", "").strip().replace("%", "")
        elif line.startswith("Missing Keywords:"):
            keywords = line.replace("Missing Keywords:", "").strip()
        elif line.startswith("Final Evaluation:"):
            summary = line.replace("Final Evaluation:", "").strip()
    try:
        match_percent = float(match)
    except:
        match_percent = 0

    results.append({
        "filename": pdf_filename,
        "match": match_percent,
        "missing": keywords,
        "summary": summary
    })
results = sorted(results, key=lambda x: x["match"], reverse=True)
print("\n🎯 Ranked Resume Matches:\n")
for idx, r in enumerate(results, 1):
    print(f" Rank {idx}: {r['filename']}")
    print(f" Match: {r['match']}%")
    print(f" Missing Keywords: {r['missing']}")
    print(f" Evaluation: {r['summary']}\n")


📤 Upload 3 resumes (one by one)...

📥 Upload resume 1:


Saving image.pdf to image.pdf

📥 Upload resume 2:


Saving Natural-Language-Processing-Nlp-Engineer-Resume-Example-730x1024.pdf to Natural-Language-Processing-Nlp-Engineer-Resume-Example-730x1024 (1).pdf

📥 Upload resume 3:


Saving IMG-20250724-WA0004.pdf to IMG-20250724-WA0004 (17).pdf

📝 Enter the job description once:
Job Description: machine language

🔍 Evaluating: image.pdf


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



🔍 Evaluating: Natural-Language-Processing-Nlp-Engineer-Resume-Example-730x1024 (1).pdf


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



🔍 Evaluating: IMG-20250724-WA0004 (17).pdf


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



🎯 Ranked Resume Matches:

🔹 Rank 1: image.pdf
   📊 Match: 1.0%
   ❌ Missing Keywords: machine language
   📝 Evaluation: The resume does not contain the required keywords "machine language."

🔹 Rank 2: Natural-Language-Processing-Nlp-Engineer-Resume-Example-730x1024 (1).pdf
   📊 Match: 0.0%
   ❌ Missing Keywords: machine language
   📝 Evaluation: The resume does not mention "machine language", so it gets a 0% match.

🔹 Rank 3: IMG-20250724-WA0004 (17).pdf
   📊 Match: 0.0%
   ❌ Missing Keywords: machine, language
   📝 Evaluation: The resume does not contain any keywords from the job description.

